#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [7]:
learning_rate = 1e-3       
batch_size = 32           
pca_variance = 0.94       # PCA varyans koruma oranı
roi_crop_size = (256,256)
patch_size = (128,128)
mlp_hidden_units = [128, 528]
mlp_dropout_rate = 0.5
mlp_epochs = 40
mlp_earlystop_patience = 25
pixels_per_cell=(16,16)
cells_per_block=(2,2)
orientations=9

In [8]:
import os, re, joblib
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, greycomatrix, greycoprops
from skimage.filters import threshold_otsu, median
from skimage.morphology import opening, disk
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from joblib import Parallel, delayed
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [9]:
def process_single_image(args):
    img_path, label, classes = args
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, roi_crop_size)
    img = img.astype(np.float32)

    center = np.array(roi_crop_size) // 2
    ps = np.array(patch_size) // 2
    patch = img[center[0]-ps[0]:center[0]+ps[0], center[1]-ps[1]+0:center[1]+ps[1]]

    patch = median(patch)
    patch = cv2.GaussianBlur(patch, (3, 3), 0)
    patch = patch.astype(np.float32) / 255.0

    return patch, classes.index(label)


In [10]:
def extract_and_preprocess_parallel(root_dir, classes, n_jobs=-1):
    class_dirs = {
        'normal': ['Birad1'],
        'benign': ['Birad3'],
        'cancer': ['Birad4', 'Birad5'],
    }
    paths, labels = [], []
    for cls, subdirs in class_dirs.items():
        for sd in subdirs:
            dir_path = os.path.join(root_dir, sd)
            if not os.path.isdir(dir_path):
                continue
            for fname in sorted(os.listdir(dir_path)):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    paths.append(os.path.join(dir_path, fname))
                    labels.append(cls)
    

    # === Paralel işleme ===
    arg_list = [(img_path, lbl, classes) for img_path, lbl in zip(paths, labels)]
    results = Parallel(n_jobs=n_jobs)(delayed(process_single_image)(args) for args in arg_list)

    X, y = zip(*results)
    return np.array(X), np.array(y)

In [11]:
def extract_features_from_image(im):
    h = hog(im, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                orientations=orientations)
    g = greycomatrix((im*255).astype(np.uint8), [1], [0], levels=256)
    glcm = [greycoprops(g, prop)[0, 0] for prop in ('contrast', 'homogeneity', 'energy')]
    area = np.sum(im > 0.5)
    perimeter = np.sum(cv2.Canny((im * 255).astype(np.uint8), 50, 150) > 0)
    ecc = 0
    if area > 10:
        pts = np.argwhere(im > 0.5)
        rect = cv2.minAreaRect(pts.astype(np.float32))
        ecc = rect[1][0] / (rect[1][1] + 1e-8)
    return np.hstack([h, glcm, area, perimeter, ecc])

def extract_features(X, out_prefix=None, n_jobs=-1):
    # 2. UPDATE THE FUNCTION NAME INSIDE THE PARALLEL CALL HERE
    feats = Parallel(n_jobs=n_jobs)(delayed(extract_features_from_image)(im) for im in X)
    feats = np.array(feats)
    pca = PCA(n_components=pca_variance)
    X_feat = pca.fit_transform(feats)
    return X_feat, pca

 Metirkler**

In [12]:
if __name__ == '__main__':
    classes = ['normal','benign','cancer']
    kok_dizin = os.path.join(os.getcwd(), "Project2")
    X_test, y_test = extract_and_preprocess_parallel(kok_dizin, classes,n_jobs=-1)
    X_feat, pca = extract_features(X_test)
    model = load_model("MLP_1_2.h5")
    T_m = 0.40
    T_b = 0.40
    probs = model.predict(X_feat)
    preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
    print("\n --------------MLP RESULTS----------------")
    print(classification_report(y_test, preds, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds, average='macro'))
    print("Recall:", recall_score(y_test, preds, average='macro'))
    print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
    try:
        print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
    except:
        print("AUC hesaplanamadı.")
    


 --------------MLP RESULTS----------------
              precision    recall  f1-score   support

      normal       0.33      1.00      0.50        20
      benign       0.00      0.00      0.00        20
      cancer       0.00      0.00      0.00        20

    accuracy                           0.33        60
   macro avg       0.11      0.33      0.17        60
weighted avg       0.11      0.33      0.17        60

Confusion Matrix:
 [[20  0  0]
 [20  0  0]
 [20  0  0]]
Accuracy: 0.3333333333333333
Precision: 0.1111111111111111
Recall: 0.3333333333333333
📌 F1 Score: 0.16666666666666666
AUC: 0.445


c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [13]:
model = joblib.load("RF_2_1.pkl")
T_m = 0.55
T_b = 0.40
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
probs = model.predict_proba(X_feat) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
print("\n --------------RF RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------RF RESULTS----------------
              precision    recall  f1-score   support

      normal       0.31      0.80      0.44        20
      benign       0.25      0.10      0.14        20
      cancer       0.00      0.00      0.00        20

    accuracy                           0.30        60
   macro avg       0.19      0.30      0.20        60
weighted avg       0.19      0.30      0.20        60

Confusion Matrix:
 [[16  4  0]
 [18  2  0]
 [18  2  0]]
Accuracy: 0.3
Precision: 0.1858974358974359
Recall: 0.3
📌 F1 Score: 0.19576719576719578
AUC: 0.4547916666666667


c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [ ]:
loaded_xgb_model_booster = xgb.Booster() 
loaded_xgb_model_booster.load_model("XGBoost_2_1.json") 
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}


dtest_predict = xgb.DMatrix(X_feat) 
probs = loaded_xgb_model_booster.predict(dtest_predict) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
T_m = 0.40
T_b = 0.55

print("\n --------------XP-B RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------XP-B RESULTS----------------
              precision    recall  f1-score   support

      normal       0.37      1.00      0.54        20
      benign       0.50      0.15      0.23        20
      cancer       0.00      0.00      0.00        20

    accuracy                           0.38        60
   macro avg       0.29      0.38      0.26        60
weighted avg       0.29      0.38      0.26        60

Confusion Matrix:
 [[20  0  0]
 [17  3  0]
 [17  3  0]]
Accuracy: 0.38333333333333336
Precision: 0.29012345679012347
Recall: 0.3833333333333333
📌 F1 Score: 0.2571032571032571
AUC: 0.6466666666666666


c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)